Imported down all the necessary libraries which is thankfully just Pandas

In [1]:
import pandas as pd

Read the dataset using Pandas read_csv feature

In [2]:
dataset=pd.read_csv('bbb_odi_data.csv')

C:\Users\Vijay\AppData\Local\Temp\ipykernel_8408\2538919985.py:1: DtypeWarning: Columns (41,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset=pd.read_csv('bbb_odi_data.csv')


Filtered out the dataset to get the instances only where the Current run rate was <=5 during the course of the game

In [3]:
dataset=dataset[dataset['inns']==2]
resultdf=dataset[(dataset['inns_rrr']<6.00)]

An unnecessary column has been deleted

In [4]:
# resultdf.drop(columns='Unnamed: 0',index=1,inplace=True)

There are several columns and I chose only the ones meaningful to the related query by using a list.

In [5]:
req_cols=['bat','batruns','ballfaced']

Now, my dataset is filtered out to get only the required columns

In [6]:
result=resultdf[req_cols]

Grouped sum of the required columns gives us the overall aggregate of the data. I've reset the columns as well to include the names as a row value in the dataset.

In [7]:
grouped=result.groupby(['bat']).sum().reset_index()

In [8]:
grouped

,bat,batruns,ballfaced
0,AB de Villiers,2569,2815
1,Aarif Sheikh,267,529
2,Aaron Finch,1440,1523
3,Aaron Hardie,2,5
4,Aaron Johnson,97,85
...,...,...,...
1241,Ziaur Rahman,12,12
1242,Zubin Surkari,100,202
1243,Zulfiqar Babar,12,7
1244,Zulqarnain Haider,5,4


I've used this different approach to sum the dismissals because there are some run out dismissals and not always it has been the striker who has gotten out, so based on the who_out? column values summation, I've found out the number of dismissals in this specific filter. 

In [9]:
dis=pd.DataFrame(resultdf['who_out?'].value_counts()).reset_index()

This is an attempt to make a similar column in both the dataframes, and use it for joins later.

In [10]:
dis.rename({'index':'bat'},axis=1,inplace=True)

In [11]:
dis

,bat,who_out?
0,Virat Kohli,73
1,Rohit Sharma,67
2,Tamim Iqbal,64
3,Mohammad Hafeez,63
4,Chris Gayle,62
...,...,...
1032,Rory McCann,1
1033,Matt Henry,1
1034,Dominic Wesonga,1
1035,Shem Ngoche,1


A left join has been performed on the left dataset which is the grouped data and then they've been joined on 'bat' column

In [12]:
finalresult=pd.merge(grouped,dis,how='left',on='bat')

Now, they've been sorted by descending order of 'batruns' and this operation has been performed in place to reflect the changes

In [13]:
finalresult.sort_values(by='batruns',ascending=False,inplace=True)

The resultant column names have been renamed for easier interpretation

In [14]:
finalresult.rename({'who_out?':'dismissals','batruns':'runs','ballfaced':'balls'},axis=1,inplace=True)

In [15]:
finalresult

,bat,runs,balls,dismissals
1203,Virat Kohli,4593,5086,73.0
934,Rohit Sharma,3825,4311,67.0
690,Martin Guptill,2706,2866,49.0
1115,Tamim Iqbal,2632,3259,64.0
0,AB de Villiers,2569,2815,42.0
...,...,...,...,...
315,Dom Joyce,0,1,1.0
316,Dominic Blampied,0,1,1.0
1059,Simandeep Singh,0,7,1.0
534,Johan van der Wath,0,1,NaN


Before performing mathematical operations to find out the averages and strike rates, I've actually verified if the data types are suitable for calculations or not. Since both are of the same 'integer' type, it is easier for me to perform the operations.

In [16]:
finalresult.dtypes

bat            object
runs            int64
balls           int64
dismissals    float64
dtype: object

The averages and Runs Per Over (rpo) were calculated from the existing data and were rounded off to the 2 nearest decimal places.

In [17]:
finalresult['avg']=(finalresult['runs']/finalresult['dismissals'])
finalresult['avg']=round(finalresult['avg'],2)

In [18]:
finalresult['rpo']=(finalresult['runs']*6/finalresult['balls'])
finalresult['rpo']=round(finalresult['rpo'],2)

The end result has been saved to a csv file to use them further for visualizations on tableau

In [19]:
finalresult.to_csv('below6.csv')

In [20]:
finalresult

,bat,runs,balls,dismissals,avg,rpo
1203,Virat Kohli,4593,5086,73.0,62.92,5.42
934,Rohit Sharma,3825,4311,67.0,57.09,5.32
690,Martin Guptill,2706,2866,49.0,55.22,5.67
1115,Tamim Iqbal,2632,3259,64.0,41.12,4.85
0,AB de Villiers,2569,2815,42.0,61.17,5.48
...,...,...,...,...,...,...
315,Dom Joyce,0,1,1.0,0.00,0.00
316,Dominic Blampied,0,1,1.0,0.00,0.00
1059,Simandeep Singh,0,7,1.0,0.00,0.00
534,Johan van der Wath,0,1,NaN,NaN,0.00
